In [27]:
# Install tfx and kfp Python packages.
import sys
!{sys.executable} -m pip install --user --upgrade -q tfx==0.23.0
!{sys.executable} -m pip install --user --upgrade -q kfp==1.0.0
# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold && mv skaffold /home/jupyter/.local/bin/

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

google-cloud-bigquery 1.24.0 requires google-resumable-media<0.6dev,>=0.5.0, but you'll have google-resumable-media 0.6.0 which is incompatible.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 44.9M  100 44.9M    0     0   105M      0 --:--:-- --:--:-- --:--:--  105M


In [28]:
!python3 -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

TFX version: 0.23.0


In [29]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin


In [30]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

ENDPOINT='https://4bab28b46f132d8f-dot-us-central2.pipelines.googleusercontent.com' 

env: GOOGLE_CLOUD_PROJECT=advert-proj
GCP project ID:advert-proj


In [31]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/advert-pred'

In [32]:
import os

#set base path and working directory
BASE_PATH = str(os.path.join(os.getcwd(), 'advert-pred'))
%cd {BASE_PATH}

[Errno 2] No such file or directory: '/home/jupyter/advert-pred/advert-pred'
/home/jupyter/advert-pred


## EXPERIMENTATION ANALYSIS

In [6]:
import pprint
import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [9]:
data_root = 'data'
data_filepath = os.path.join(data_root, "advertising.csv")

!head {data_filepath}

DailyTimeSpentOnSite,Age,AreaIncome,DailyInternetUsage,AdTopicLine,City,Male,Country,Timestamp,ClickedOnAd
68.95,35,61833.9,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,2016-03-27 00:53:11,0
80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,2016-04-04 01:39:02,0
69.47,26,59785.94,236.5,Organic bottom-line service-desk,Davidton,0,San Marino,2016-03-13 20:35:42,0
74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,2016-01-10 02:31:19,0
68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,2016-06-03 03:36:18,0
59.99,23,59761.56,226.74,Sharable client-driven software,Jamieberg,1,Norway,2016-05-19 14:30:17,0
88.91,33,53852.85,208.36,Enhanced dedicated support,Brandonstad,0,Myanmar,2016-01-28 20:59:32,0
66.0,48,24593.33,131.76,Reactive local challenge,Port Jefferybury,1,Australia,2016-03-07 01:40:15,1
74.53,30,68862.0,221.51,Configurable coherent function,West Colin,1,Grenada,2016-04-18 

In [8]:
context = InteractiveContext()

In [9]:
example_gen = CsvExampleGen(input=external_input(data_root))
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/CsvExampleGen/examples/1, id: 2)]
        ))

In [10]:
# %%skip_for_export

artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/CsvExampleGen/examples/1


In [11]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/StatisticsGen/statistics/2, id: 3)]
        ))

In [12]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [13]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'AdTopicLine',BYTES,required,single,-
'City',BYTES,required,single,-
'Country',BYTES,required,single,-
'Timestamp',BYTES,required,single,-
'Age',INT,required,single,-
'AreaIncome',FLOAT,required,single,-
'ClickedOnAd',INT,required,single,-
'DailyInternetUsage',FLOAT,required,single,-
'DailyTimeSpentOnSite',FLOAT,required,single,-


In [14]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

In [15]:
advert_transform = 'model/advert-transform.py'

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=advert_transform)

context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
Use ref() instead.


  name: "AdTopicLine"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "City"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Country"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Timestamp"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Age"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "AreaIncome"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "ClickedOnAd"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  bool_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Transform/transform_graph/5/.temp_path/tftransform_tmp/40c01070c97442459d24e82ef94f3ac8/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_

  name: "AdTopicLine"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "City"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Country"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Timestamp"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Age"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "AreaIncome"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "ClickedOnAd"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  bool_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1


  name: "Age"
  type: INT
}
feature {
  name: "AreaIncome"
  type: FLOAT
}
feature {
  name: "City"
  type: BYTES
}
feature {
  name: "ClickedOnAd"
  type: INT
}
feature {
  name: "Country"
  type: BYTES
}
feature {
  name: "DailyInternetUsage"
  type: FLOAT
}
feature {
  name: "DailyTimeSpentOnSite"
  type: FLOAT
}
feature {
  name: "Male"
  type: INT
}
}), (<PCollection[Analyze/CreateSavedModel/BindTensors/ReplaceWithConstants.None] at 0x7f0d8c5d9290>, BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "Age_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "AreaIncome_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "City_xf"
  type: INT
  int_domain {
    is_categorical: true
  }
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "ClickedOnAd_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "Country_xf"
  type: INT
  int_dom

  name: "Age"
  type: INT
}
feature {
  name: "AreaIncome"
  type: FLOAT
}
feature {
  name: "City"
  type: BYTES
}
feature {
  name: "ClickedOnAd"
  type: INT
}
feature {
  name: "Country"
  type: BYTES
}
feature {
  name: "DailyInternetUsage"
  type: FLOAT
}
feature {
  name: "DailyTimeSpentOnSite"
  type: FLOAT
}
feature {
  name: "Male"
  type: INT
}
}), (<PCollection[Analyze/CreateSavedModel/BindTensors/ReplaceWithConstants.None] at 0x7f0d8c5d9290>, BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "Age_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "AreaIncome_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "City_xf"
  type: INT
  int_domain {
    is_categorical: true
  }
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "ClickedOnAd_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "Country_xf"
  type: INT
  int_dom

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Transform/transform_graph/5/.temp_path/tftransform_tmp/618ebc8b3eac4911ada7dbc918f2e136/assets
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Transform/transform_graph/5/.temp_path/tftransform_tmp/618ebc8b3eac4911ada7dbc918f2e136/saved_model.pb
value: "\n\013\n\tConst_3:0\022-vocab_compute_and_apply_vocabulary_vocabulary"

value: "\n\013\n\tConst_5:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"

value: "\n\013\n\tConst_7:0\022/vocab_compute_and_apply_vocabulary_2_vocabulary"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
value: "\n\013\n\tConst_3:0\022-vocab_compute_and_apply_vocabula

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Transform/transformed_examples/5, id: 7)]
        ))

In [16]:
transform.outputs

{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Transform/transform_graph/5, id: 6)]
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Transform/transformed_examples/5, id: 7)]
)}

In [17]:
advert_trainer = 'model/advert-trainer.py'

trainer = Trainer(
    module_file=advert_trainer,
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500))

context.run(trainer)

   1/1000 [..............................] - ETA: 0s - loss: 0.7108 - binary_accuracy: 0.2500WARNING:tensorflow:From /home/jupyter/.local/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1000/1000 [==============================] - 4s 4ms/step - loss: 0.0524 - binary_accuracy: 0.9859 - val_loss: 0.0983 - val_binary_accuracy: 0.9772
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Trainer/model/6/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Trainer/model/6, id: 8)]
        ))

In [22]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='ClickedOnAd')],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ])

model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 10
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Trainer/model/6, id: 8)]
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(type_name: ModelBlessing, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Evaluator/blessing/8, id: 10)]
        ))

In [23]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 11
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Evaluator/evaluation/11, id: 11)]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(type_name: ModelBlessing, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Evaluator/blessing/11, id: 12)]
        ))

In [24]:
evaluator.outputs

{'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: [Artifact(type_name: ModelEvaluation, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Evaluator/evaluation/11, id: 11)]
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: [Artifact(type_name: ModelBlessing, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Evaluator/blessing/11, id: 12)]
)}

In [25]:
context.show(evaluator.outputs['evaluation'])

In [26]:
blessing_uri = evaluator.outputs.blessing.get()[0].uri
!ls -l {blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Sep  8 13:30 BLESSED


In [28]:
serving_model_dir = 'serving_model/advert-pred'

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=serving_model_dir)))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 12
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(type_name: PushedModel, uri: /tmp/tfx-interactive-2020-09-08T13_29_00.216206-4_736tpz/Pusher/pushed_model/12, id: 13)]
        ))

### SET UP KUBEFLOW PIPELINE

In [13]:
## copy data to cloud storage for easy access from Kubeflow
!gsutil cp data/advertising.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/advert-pred/data/advertising.csv

Copying file://data/advertising.csv [Content-Type=text/csv]...
/ [1 files][104.9 KiB/104.9 KiB]                                                
Operation completed over 1 objects/104.9 KiB.                                    


In [35]:
PIPELINE_NAME = 'adclicked_pipeline'

In [36]:
!tfx pipeline create  \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

2020-09-08 17:28:49.185002: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2020-09-08 17:28:49.185169: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
Target image gcr.io/advert-proj/advert-pred is not used. If the build spec is provided, update the target image in the build spec file build.yaml.
[Skaffold] Generating tags...
[Skaffold]  - gcr.io/advert-proj/advert-pred-pipeline -> gcr.io/advert-proj/advert-pred-pipeline:latest
[Skaffold] Checking cache...
[Skaffold]  - gcr.io/advert-proj/advert-pred-pipeline: Not found.

In [37]:
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

2020-09-08 17:29:40.809988: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2020-09-08 17:29:40.812436: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Creating a run for pipeline: adclicked_pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: adclicked_pipeline
+--------------------+--------------------------------------+----------+---------------------------+------------------------------------------------------------------------------------------------------------------------------+
| pipeline_name      | run_id                               | status   | created_at                | link  

In [38]:
!tfx pipeline update \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT}

!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

2020-09-08 17:32:08.254768: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2020-09-08 17:32:08.254941: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
[Skaffold] Generating tags...
[Skaffold]  - gcr.io/advert-proj/advert-pred-pipeline -> gcr.io/advert-proj/advert-pred-pipeline:latest
[Skaffold] Checking cache...
[Skaffold]  - gcr.io/advert-proj/advert-pred-pipeline: Not found. Building
[Skaffold] Building [gcr.io/advert-proj/advert-pred-pipeline]...
Sending build context to Docker daemon  2.363MBon  524.8kB
[Skaffold] St